# The Complete LangChain and LLM Guide
## Module 4: LangChain Prompt Templates

In [1]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

### ChatOpenAI - with fString

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
def get_openai_key():
    load_dotenv(find_dotenv())
    key = os.getenv("OPENAI_API_KEY")
    if not key:
        raise EnvironmentError("OPENAI_API_KEY is missing.")
    return key

In [4]:
openai_api_key = get_openai_key()

#### OpenAI Model API Prices    

| Model               | price per 1M Input Tokens | price per 1M Output Tokens | Characteristics / Notes  |    
---------------------|------------------------|--------------------------|----------------------------------------------------------------------------------------|    
| **GPT-5 nano**      | 0.05                   | 0.40                     | Fastest, cheapest GPT-5 variant; ideal for summarization/classification  |
| **GPT-4o mini**     | 0.15                   | 0.60                     | Multimodal (text, image, audio); cheaper than gpt-3.5; smaller and faster variant  |    
| **GPT-5 mini**      | 0.25                   | 2.00                     | Mid-tier GPT-5 variant, balanced speed and cost  |
| **gpt-3.5-turbo**   | 0.50                   | 1.50                     | Highly cost-effective for general chat and coding; 16k context window  |     
| **GPT-5 (full)**    | 1.25                   | 10.00                    | Flagship GPT-5: high performance, large 400k+ context window |
| **GPT-4o (full)**   | 2.50                   | 10.00                    | Powerful multimodal model (vision, speech, translation); high performance  |
| **GPT-4.5**         | 75.00                  | 150.00                   | Very large model, creative insights, training via unsupervised learning; expensive |
| **o1-pro**          | 150.00                 | 600.00                   | Top-tier “reasoning” model; highest cost yet, powerful STEM capability |


In [5]:
llm_model = "gpt-5-nano"

In [6]:
chat_llm = ChatOpenAI(api_key=openai_api_key, model = llm_model)

In [7]:
type(chat_llm)

langchain_openai.chat_models.base.ChatOpenAI

In [8]:
customer_review = """
    I can't believe I wasted my time and money on this pathetic excuse for a service. 
    This is a scam, and anyone who buys from you is an idiot. 
    You people are a joke.
"""

In [9]:
template = f"""
    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.
    Review: {customer_review}
"""
template

"\n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"

In [10]:
# create the message list 
messages = [{"role":"user", "content" : template}]
messages

[{'role': 'user',
  'content': "\n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Portuguese.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"}]

In [11]:
response = chat_llm.invoke(messages)
print(response)

content='- Polite English version:\nRegrettably, I feel that my time and money were not well spent on this service. The experience did not meet my expectations, and I am concerned that it did not align with the claims advertised. I would appreciate assistance with a satisfactory resolution, such as a refund or guidance on how to resolve the issue. Thank you for your attention to this matter.\n\n- Portuguese translation:\nInfelizmente, sinto que meu tempo e meu dinheiro não foram bem gastos neste serviço. A experiência não atendeu às minhas expectativas e temo que não tenha correspondido às promessas anunciadas. Agradeceria assistência para uma resolução satisfatória, como um reembolso ou orientações sobre como resolver o problema. Agradeço pela atenção a este assunto.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2274, 'prompt_tokens': 76, 'total_tokens': 2350, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 

### ChatAPI - with Prompt Templates

#### Explanation of ChatPromptTemplate.from_template (from Gemini)

`ChatPromptTemplate.from_template` is a convenient way to create a `ChatPromptTemplate` in LangChain from a simple string template. It automatically handles the conversion of a single string into a list of messages, assuming the user's input is a single message.

The main benefit of using `from_template` is its simplicity for common use cases. You don't need to manually define the `system` and `human` messages as separate components. The method takes a single string, and it automatically creates a `HumanMessagePromptTemplate` from it.

Here's a breakdown of how it works and a simple example:

**How it works:**

1.  **Input:** You provide a single string that acts as a template for the human message. You can include placeholders (e.g., `{input_variable}`) that will be filled in later.

2.  **Automatic Conversion:** LangChain's `from_template` method takes this string and creates a `HumanMessagePromptTemplate` object. This object represents a message from the user.

3.  **Result:** The final output is a `ChatPromptTemplate` with a single message template inside it. When you format this template with your input variables, it produces a list containing a single `HumanMessage` object.

**Example:**

Let's say you want to create a simple prompt that asks a user for a company name and then suggests a catchy slogan.

```python
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

# Define the template string
template = "What is a good slogan for a company named {company_name}?"

# Create the ChatPromptTemplate using from_template
prompt = ChatPromptTemplate.from_template(template)

# Now, let's use the prompt
input_variables = {"company_name": "Acme Inc."}
formatted_messages = prompt.format_messages(**input_variables)

# The result is a list of Chat messages
print(formatted_messages)
print(type(formatted_messages[0]))
```

**Output:**

```
[HumanMessage(content='What is a good slogan for a company named Acme Inc.')]
<class 'langchain_core.messages.human.HumanMessage'>
```

**Key Takeaways:**

  * **Simplicity:** Best for simple, single-message prompts.
  * **Automatic `HumanMessage`:** It automatically wraps your template string in a `HumanMessagePromptTemplate`.
  * **Alternative to `from_messages`:** It's a quick alternative to `ChatPromptTemplate.from_messages` **when you only need a single user message**.
  * **Limitations:** For more complex prompts that require multiple message types (e.g., `SystemMessage`, `AIMessage`, or a series of `HumanMessage`s), you should use `ChatPromptTemplate.from_messages`.

**When to use `from_template`:**

  * You're building a simple chatbot or tool that takes a single user query.
  * You want a quick and easy way to create a `ChatPromptTemplate` without a lot of boilerplate code.
  * Your prompt only consists of a single `HumanMessage`.

**In summary**, `ChatPromptTemplate.from_template` is your go-to function for creating a simple, single-message prompt in LangChain.

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
llm_model = "gpt-5-nano"

In [14]:
chat_llm = ChatOpenAI(api_key=openai_api_key, model = llm_model)

In [15]:
customer_review = """
    I can't believe I wasted my time and money on this pathetic excuse for a service. 
    This is a scam, and anyone who buys from you is an idiot. 
    You people are a joke.
"""

In [16]:
# Define the template string
template = f"""
    Rewrite the customer review message in a polite tone, and then please translate the new review message into Italian.
    Review: {customer_review}
"""

In [17]:
# Create the ChatPromptTemplate using from_template
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="\n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Italian.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"), additional_kwargs={})])

In [18]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [19]:
# Format the prompt with the input variable, which creates a LangChain message object
formatted_prompt = prompt.format(customer_review=customer_review)
formatted_prompt

"Human: \n    Rewrite the customer review message in a polite tone, and then please translate the new review message into Italian.\n    Review: \n    I can't believe I wasted my time and money on this pathetic excuse for a service. \n    This is a scam, and anyone who buys from you is an idiot. \n    You people are a joke.\n\n"

In [20]:
type(formatted_prompt)

str

In [21]:
# Directly invoke the language model with the formatted prompt
response = chat_llm.invoke(formatted_prompt)

In [22]:
type(response)

langchain_core.messages.ai.AIMessage

In [23]:
# The result is a list of Chat messages
print(response)
print("---------------------")
print(response.content)

content='Polite rewrite (English):\nI am disappointed with the service and feel that my time and money were not well spent. The experience did not meet my expectations, and I would not recommend this company to others. I hope you take customer feedback seriously and consider making improvements.\n\nItalian translation:\nSono deluso dal servizio e sento che il mio tempo e i miei soldi non sono stati ben spesi. Questa esperienza non ha soddisfatto le mie aspettative e non consiglierei questa azienda ad altri. Spero che prendiate sul serio il feedback dei clienti e che valutiate eventuali miglioramenti.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1282, 'prompt_tokens': 78, 'total_tokens': 1360, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1152, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 's

#### Variant using chains

In [24]:
prompt = ChatPromptTemplate.from_template(template)

In [25]:
chain = prompt | chat_llm

In [26]:
input_variables = {"customer_review": customer_review}

In [27]:
response = chain.invoke(input_variables)

In [28]:
print(response)
print("---------------------")
print(response.content)

content='Polite rewrite (English):\nRegrettably, I am disappointed with the service and feel that it did not provide good value for the money I spent. I felt misled by how the product or service was described, and I would appreciate a clear explanation or resolution. I hope you can address these concerns so that other customers have a better experience.\n\nItalian translation:\nCon rammarico, sono deluso dal servizio e ritengo che non abbia offerto un buon rapporto qualità-prezzo rispetto al costo sostenuto. Mi è sembrato che la descrizione non corrispondesse a quanto offerto, e apprezzerei una spiegazione chiara o una soluzione. Spero che possiate affrontare queste preoccupazioni in modo che anche gli altri clienti abbiano una migliore esperienza.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2346, 'prompt_tokens': 76, 'total_tokens': 2422, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens